# Example: Predictions for HRES T0

In this example, we will download HRES T0 data for 11 May 2022 from [WeatherBench2](https://weatherbench2.readthedocs.io/) at 0.25 degrees resolution and run Aurora on this data. We will use the version of Aurora that was fine-tuned on IFS HRES T0 in this example.

Running this notebook requires additional Python packages. You can install these as follows:

```
pip install gcsfs cdsapi xarray zarr netcdf4 matplotlib
```

## Downloading the Data

To begin with, we download the data from [WeatherBench2](https://weatherbench2.readthedocs.io/).

In [ ]:
# from pathlib import Path

# import fsspec
# import xarray as xr

# # Day to download. This will download all times for that day.
# day = "2022-05-11"

# # Data will be downloaded here.
# download_path = Path(f"/Users/pmpaquet/Projects/Stanford/CS229S/Project/Project_aurora/data/{day}")

# download_path = download_path.expanduser()
# download_path.mkdir(parents=True, exist_ok=True)

# # We will download from Google Cloud.
# url = "gs://weatherbench2/datasets/hres_t0/2016-2022-6h-1440x721.zarr"
# ds = xr.open_zarr(fsspec.get_mapper(url), chunks=None)

# # Download the surface-level variables. We write the downloaded data to another file to cache.
# if not (download_path / f"{day}-surface-level.nc").exists():
#     surface_vars = [
#         "10m_u_component_of_wind",
#         "10m_v_component_of_wind",
#         "2m_temperature",
#         "mean_sea_level_pressure",
#     ]
#     ds_surf = ds[surface_vars].sel(time=day).compute()
#     ds_surf.to_netcdf(str(download_path / f"{day}-surface-level.nc"))
# print("Surface-level variables downloaded!")

# # Download the atmospheric variables. We write the downloaded data to another file to cache.
# if not (download_path / f"{day}-atmospheric.nc").exists():
#     atmos_vars = [
#         "temperature",
#         "u_component_of_wind",
#         "v_component_of_wind",
#         "specific_humidity",
#         "geopotential",
#     ]
#     ds_atmos = ds[atmos_vars].sel(time=day).compute()
#     ds_atmos.to_netcdf(str(download_path / f"{day}-atmospheric.nc"))
# print("Atmos-level variables downloaded!")

Surface-level variables downloaded!
Atmos-level variables downloaded!


## Downloading Static Variables from ERA5 Data

The static variables are not available in WeatherBench2, so we need to download them from ERA5, just like we did [in the example for ERA5](example_era5.ipynb).
To do so, register an account with the [Climate Data Store](https://cds-beta.climate.copernicus.eu/) and create `$HOME/.cdsapirc` with the following content:

```
url: https://cds-beta.climate.copernicus.eu/api
key: <API key>
```

You can find your API key on your account page.

In order to be able to download ERA5 data, you need to accept the terms of use in the [dataset page](https://cds-beta.climate.copernicus.eu/datasets/reanalysis-era5-single-levels?tab=download).

In [9]:
from pathlib import Path

import cdsapi

# Data will be downloaded here.
download_path = Path("/Users/pmpaquet/Projects/Stanford/CS229S/Project/Project_aurora/data")

c = cdsapi.Client()

download_path = download_path.expanduser()
download_path.mkdir(parents=True, exist_ok=True)

# Download the static variables.
if not (download_path / "static.nc").exists():
    c.retrieve(
        "reanalysis-era5-single-levels",
        {
            "product_type": "reanalysis",
            "variable": [
                "geopotential",
                "land_sea_mask",
                "soil_type",
            ],
            "year": "2023",
            "month": "01",
            "day": "01",
            "time": "00:00",
            "format": "netcdf",
        },
        str(download_path / "static.nc"),
    )
print("Static variables downloaded!")

2024-11-30 13:34:42,933 WARNING [2024-11-27T00:00:00] System is back on service under testing conditions. Please follow updates [here](https://forum.ecmwf.int/t/cds-ads-and-ewds-down-until-further-notice/8015) and status [here](https://status.ecmwf.int/)
2024-11-30 13:34:42,944 INFO [2024-09-28T00:00:00] **Welcome to the New Climate Data Store (CDS)!** This new system is in its early days of full operations and still undergoing enhancements and fine tuning. Some disruptions are to be expected. Your 
[feedback](https://jira.ecmwf.int/plugins/servlet/desk/portal/1/create/202) is key to improve the user experience on the new CDS for the benefit of everyone. Thank you.
2024-11-30 13:34:42,946 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2024-11-30 13:34:42,968 INFO [2024-09-16T00:00:00] Remember that you need to have an ECMWF account to use the new CDS. **Your old CDS credentials will not work in new CDS!**
2024-

Static variables downloaded!


# Download loop

In [10]:
from pathlib import Path

import fsspec
import xarray as xr

def download_for_day(day:str) -> None:
    # Data will be downloaded here.
    download_path = Path("/Users/pmpaquet/Projects/Stanford/CS229S/Project/Project_aurora/data")
    download_path = download_path / day

    download_path = download_path.expanduser()
    download_path.mkdir(parents=True, exist_ok=True)

    # We will download from Google Cloud.
    url = "gs://weatherbench2/datasets/hres_t0/2016-2022-6h-1440x721.zarr"
    ds = xr.open_zarr(fsspec.get_mapper(url), chunks=None)

    # Download the surface-level variables. We write the downloaded data to another file to cache.
    if not (download_path / f"{day}-surface-level.nc").exists():
        surface_vars = [
            "10m_u_component_of_wind",
            "10m_v_component_of_wind",
            "2m_temperature",
            "mean_sea_level_pressure",
        ]
        ds_surf = ds[surface_vars].sel(time=day).compute()
        ds_surf.to_netcdf(str(download_path / f"{day}-surface-level.nc"))
    print("\tSurface-level variables downloaded!")

    # Download the atmospheric variables. We write the downloaded data to another file to cache.
    if not (download_path / f"{day}-atmospheric.nc").exists():
        atmos_vars = [
            "temperature",
            "u_component_of_wind",
            "v_component_of_wind",
            "specific_humidity",
            "geopotential",
        ]
        ds_atmos = ds[atmos_vars].sel(time=day).compute()
        ds_atmos.to_netcdf(str(download_path / f"{day}-atmospheric.nc"))
    print("\tAtmos-level variables downloaded!")

In [11]:
# base_days = [
#     "2022-05-11",
#     "2022-02-01",
#     "2022-07-01",
#     "2022-11-01",
# ]

base_days = [
    "2022-04-01",
    "2022-08-01",
]

n_days = 21
for base_day in base_days:
    y,m,_d = base_day.split('-')
    for i in range(n_days):
        d = f'{int(_d)+i:02}'
        if int(d) > 31:
            m = '06'
            d = f'{int(d) - 31:02}'
        day_str = f'{y}-{m}-{d}'
        print(day_str)
        download_for_day(day_str)
        print()

2022-04-01
	Surface-level variables downloaded!
	Atmos-level variables downloaded!

2022-04-02
	Surface-level variables downloaded!
	Atmos-level variables downloaded!

2022-04-03
	Surface-level variables downloaded!
	Atmos-level variables downloaded!

2022-04-04
	Surface-level variables downloaded!
	Atmos-level variables downloaded!

2022-04-05
	Surface-level variables downloaded!
	Atmos-level variables downloaded!

2022-04-06
	Surface-level variables downloaded!
	Atmos-level variables downloaded!

2022-04-07
	Surface-level variables downloaded!
	Atmos-level variables downloaded!

2022-04-08
	Surface-level variables downloaded!
	Atmos-level variables downloaded!

2022-04-09
	Surface-level variables downloaded!
	Atmos-level variables downloaded!

2022-04-10
	Surface-level variables downloaded!
	Atmos-level variables downloaded!

2022-04-11
	Surface-level variables downloaded!
	Atmos-level variables downloaded!

2022-04-12
	Surface-level variables downloaded!
	Atmos-level variables downl